In [1]:
import pandas as pd
import os
from sklearn.utils import shuffle
from tqdm import tqdm_notebook
from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification,BertAdam
tokenizer = BertTokenizer.from_pretrained("bert-large-uncased")

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
src = pd.read_csv("~/mt-dnn/toxic_data/train.csv")

In [9]:
def set_prep(sample,filename):
    with open (filename,"a") as tp:
        for i in tqdm_notebook(range(sample.shape[0])):
            tokens = tokenizer.tokenize(sample.iloc[i]["comment_text"])
            if len(tokens)>510:
                tokens = tokens[0:510]
            tokens = ['[CLS]'] + tokens + ['[SEP]']
            tokens = tokenizer.convert_tokens_to_ids(tokens)
            _id = sample.loc[i]["id"]
            target = sample.loc[i]["target"]
            _type = str([0]*len(tokens))
            line = '{{"uid": {0}, "label": {1}, "token_id": {2}, "type_id": {3}}}'.format(_id,target,tokens,_type)
            tp.write(line+'\n')

In [7]:
def toxic_prep(src,path_to_prep="/home/jupyter/mt-dnn/data/mt_dnn"):
    train = src.loc[:int(src.shape[0]*0.95),:]
    dev = src.loc[int(src.shape[0]*0.95):,:]
    set_prep(train,os.path.join(path_to_prep,"toxic_train.json"))
    set_prep(dev,os.path.join(path_to_prep,"toxic_dev.json"))

In [8]:
toxic_prep(src)

TypeError: '>' not supported between instances of 'list' and 'int'